In [60]:
import pandas as pd
import numpy as np

import yfinance as yf
import plotly.graph_objects as go

In [61]:
## Fetch Data
ticker_data = yf.download("EURNZD=X", period="3y", interval="1d")

ticker_data = ticker_data[["Open", "Close", "Low", "High"]]

[*********************100%***********************]  1 of 1 completed


In [62]:
class Backtest:
    balance = 0

    stoploss = 0
    target = 0

    position = {
        "entry_date": None,
        "entry_price": None,
        "exit_price": None,
        "exit_date": None,
        "target": None,
        "stoploss": None,
        "PnL": None,
        "crossover_type": None,
        "active": False,
        "size": None,
        "cost": None,
        "balance": None,
    }

    trade_book = pd.DataFrame(
        columns=[
            "entry_date",
            "entry_price",
            "exit_price",
            "exit_date",
            "target",
            "stoploss",
            "PnL",
            "crossover_type",
            "active",
            "size",
            "margin_used",
            "balance",
            "balance_on_open"
        ]
    )

    def __init__(self, balance, stoploss, reward_ratio):
        self.balance = balance
        self.stoploss = stoploss
        self.target = stoploss * reward_ratio        
        
        self.clear_position()
        self.reset_tradebook()

    def clear_position(self):
        self.position["entry_date"] = None
        self.position["entry_price"] = None
        self.position["exit_price"] = None
        self.position["exit_date"] = None

        self.position["target"] = None
        self.position["stoploss"] = None

        self.position["PnL"] = None
        self.position["crossover_type"] = None

        self.position["active"] = False
        self.position["size"] = 0
        self.position["balance"] = None

        self.position["status"] = None


    def reset_tradebook(self):
        self.trade_book = pd.DataFrame(
            columns=[
                "entry_date",
                "entry_price",
                "exit_price",
                "exit_date",
                "target",
                "stoploss",
                "PnL",
                "crossover_type",
                "active",
                "size",
                "margin_used",
                "balance",
                "balance_on_open"
            ]
        )
        
        
    def open(self, entry_date, entry_price, stoploss, target, size, crossover_type):
        if self.balance is None or self.balance == 0:
            raise Exception("Oops, you ran out of bucks.")
        
        if self.balance < entry_price*size:
            print("Balance: ", self.balance)
            print("Lot Size: ", size)
            print("Ticket Price: ", entry_price*size)
            raise Exception("Oops, you are low on bucks.")

        self.position["active"] = True

        self.position["entry_date"] = entry_date
        self.position["entry_price"] = entry_price

        self.position["stoploss"] = stoploss
        self.position["target"] = target

        self.position["size"] = size
        self.position["crossover_type"] = crossover_type
        self.position["margin_used"] = entry_price * size
        self.position["balance_on_open"] = self.balance - entry_price * size 

        self.balance = self.balance - entry_price * size

    def close(self, candle, exit_price, exit_date):
        # Closes the trade / position
        self.position["active"] = False
        self.position["exit_date"] = candle.name
        self.position["exit_price"] = exit_price
        self.position["PnL"] = (
            exit_price - self.position["entry_price"]
        ) * self.position["size"]
        
        if self.position["crossover_type"] == "short":
            self.position["PnL"] = -1*self.position["PnL"]

        balance = self.position['balance_on_open'] + \
            self.position['margin_used'] + self.position['PnL'] 
        self.position["balance"] = balance
        self.balance = balance

        row = pd.Series(self.position)
        self.trade_book.loc[len(self.trade_book)] = row

        print(self.position)

        self.clear_position()

    def strategy():
        ## to be implemented in subclass
        pass

    def print(self):
        print(self.balance, self.stoploss, self.target)

In [63]:
class Tester(Backtest):
    ticker_data = None
    size = 10

    def __init__(self, balance, stoploss, reward_ratio):
        super().__init__(balance, stoploss, reward_ratio)
        self.ticker_data = None
        self.reset_tradebook()
        
        self.profitable_trades = None
        self.losing_trades = None

    def init(self, ticker_data, size, small_ema, large_ema):
        ticker_data["small_ema"] = (
            ticker_data["Close"].ewm(span=small_ema, adjust=False).mean()
        )

        ticker_data["large_ema"] = (
            ticker_data["Close"].ewm(span=large_ema, adjust=False).mean()
        )

        ticker_data["crossover_pos"] = np.where(
            ticker_data["small_ema"] > ticker_data["large_ema"], 1, 0
        )

        ticker_data["crosspoint"] = np.where(
            ticker_data["crossover_pos"] != ticker_data["crossover_pos"].shift(1),
            True,
            False,
        )
        
        self.small_ema = small_ema
        self.large_ema = large_ema
        self.ticker_data = None
        self.ticker_data = ticker_data
        self.size = size

    def chart(self, small_ema, large_ema):
        fig = go.Figure(
            data=[
                go.Candlestick(
                    x=ticker_data.index,
                    open=ticker_data["Open"],
                    high=ticker_data["High"],
                    low=ticker_data["Low"],
                    close=ticker_data["Close"],
                ),
                go.Scatter(
                    x=ticker_data.index,
                    y=ticker_data["small_ema"],
                    name=f"small_ema_{small_ema}",
                ),
                go.Scatter(
                    x=ticker_data.index,
                    y=ticker_data["large_ema"],
                    name=f"large_ema_{large_ema}",
                ),
            ]
        )

        fig.update_layout(xaxis_rangeslider_visible=False)
        fig.show()

    def print(self):
        print(self.position)

    def run(self):
        if self.ticker_data is None:
            raise Exception("No ticker data to run backtest on")
        
        # Skip first 200 candles
        start_index = self.large_ema

        while start_index < len(self.ticker_data):
            self.strategy(self.ticker_data.iloc[start_index])
            start_index += 1

    def stats(self):
        print("Total Trades: ", len(self.trade_book))
        print("PnL: ", self.trade_book["PnL"].sum())

        total_trades = len(self.trade_book)
        profitable_trades = self.trade_book[self.trade_book["PnL"] > 0]
        losing_trades = self.trade_book[self.trade_book["PnL"] < 0]
        win_percentage = len(profitable_trades) / total_trades * 100
        lose_percentage = len(losing_trades) / total_trades * 100
        
        
        print("Profitable Trades: ", len(profitable_trades))
        print("Losing Trades: ", len(losing_trades))
        print("Win Percentage: ", win_percentage)
        print("Lose Percentage: ", lose_percentage)

        self.profitable_trades = profitable_trades
        self.losing_trades = losing_trades

    def strategy(self, candle):
        if self.position["active"] == True:
            # check if target met or stoploss hit
            # calculate trade results, PnL
            # close position

            if self.position["crossover_type"] == "long":
                if candle["Close"] < self.position["stoploss"]:
                    self.close(candle, self.position["stoploss"], candle.name)
                    return

                if candle["Close"] > self.position["target"]:
                    self.close(candle, self.position["target"], candle.name)
                    return

            if self.position["crossover_type"] == "short":
                if candle["Close"] > self.position["stoploss"]:
                    self.close(candle, self.position["stoploss"], candle.name)
                    return

                if candle["Close"] < self.position["target"]:
                    self.close(candle, self.position["target"], candle.name)
                    return

            return

        if candle["crosspoint"] == True:
            entry_price = candle["Open"]
            crossover_type = "long" if candle["crossover_pos"] == 1 else "short"
            stoploss = None
            target = None
            
            if crossover_type == "short":
                stoploss = entry_price + self.stoploss
                target = entry_price - self.target
                
            if crossover_type == "long":
                stoploss = entry_price - self.stoploss
                target = entry_price + self.target

            self.open(
                entry_date=candle.name,
                entry_price=entry_price,
                stoploss=stoploss,
                target=target,
                size=self.size,
                crossover_type=crossover_type,
            )

In [64]:
small_ema = 55
large_ema = 200

# balance, stoploss, reward_ratio
ttr = Tester(25_00_000, 0.01, 3)
# ticker_data, ticket_size, small_ema, large_ema
ttr.init(ticker_data, 1_00_000, small_ema, large_ema)
# small_ema, large_ema
ttr.chart(small_ema, large_ema)
# run backtesting
ttr.run()
ttr.stats()

t_book2 = ttr.trade_book
profitables = ttr.profitable_trades
losers = ttr.losing_trades

/tmp/ipykernel_30809/3485857716.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



{'entry_date': Timestamp('2022-02-07 00:00:00'), 'entry_price': 1.730239987373352, 'exit_price': 1.720239987373352, 'exit_date': Timestamp('2022-02-09 00:00:00'), 'target': 1.760239987373352, 'stoploss': 1.720239987373352, 'PnL': -1000.0000000000009, 'crossover_type': 'long', 'active': False, 'size': 100000, 'cost': None, 'balance': 2499000.0, 'status': None, 'margin_used': 173023.9987373352, 'balance_on_open': 2326976.001262665}
{'entry_date': Timestamp('2022-03-07 00:00:00'), 'entry_price': 1.581629991531372, 'exit_price': 1.591629991531372, 'exit_date': Timestamp('2022-03-09 00:00:00'), 'target': 1.551629991531372, 'stoploss': 1.591629991531372, 'PnL': -1000.0000000000009, 'crossover_type': 'short', 'active': False, 'size': 100000, 'cost': None, 'balance': 2498000.0, 'status': None, 'margin_used': 158162.9991531372, 'balance_on_open': 2340837.000846863}
{'entry_date': Timestamp('2022-07-01 00:00:00'), 'entry_price': 1.6790399551391602, 'exit_price': 1.6690399551391601, 'exit_date': 

/tmp/ipykernel_30809/4018925039.py:130: FutureWarning:

In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.

/tmp/ipykernel_30809/4018925039.py:130: FutureWarning:

In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.

/tmp/ipykernel_30809/4018925039.py:130: FutureWarning:

In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.

